# Get requests to Flemish geolocation service

https://docs.python.org/3.1/howto/urllib2.html

In [1]:
import urllib.request
import urllib.parse
import json

### Details about api if of geopunt
* http://loc.geopunt.be/Help/Api/GET-v2-Location_q_latlon_xy_capakey_poi_c

* international, have a look here: http://www.latlong.net/

In [2]:
def latlon_from_address(address):
      
    url = 'http://loc.geopunt.be/v2/Location?'
    
    #strip spaces for more robustness
    values = {'q' : address.strip()}
    
    data = urllib.parse.urlencode(values)

    #get request is url + data
    #for POST request you need data.encode('utf8') first and use binary_data as a second arg instead of a '+'
    req = urllib.request.Request(url + data)
    
    req.add_header('Content-Type', 'text/json')
    response = urllib.request.urlopen(req)
    binary_response = response.read()
    decoded = binary_response.decode('utf8')
    
    response.close()

    jsonobj = json.loads(decoded)
    
    results = jsonobj['LocationResult']
    
    if len(results) == 0:
        return None
    
    location = results[0]['Location']

    latkeyID = 'Lat_WGS84'
    lonkeyID = 'Lon_WGS84'
    
    return (location[latkeyID], location[lonkeyID])
        


In [3]:
print(latlon_from_address('Ruiselede '))

(51.06302363494435, 3.38665410746602)


In [4]:
import pandas as pd

In [5]:
all_data =  pd.read_json('EVA_restodata.json')

In [6]:
all_data.head(n=3)

,city,name,street,tags,zipcode
0,Neerijse,Biesbemd,Kamstraat 33,"[Approved by EVA, EVA voordeel, Veganvriendelijk]",3040
1,Antwerpen,Falafel Shami,Hoogstraat 47,"[Snack, 100% vegetarisch, Approved by EVA, Veg...",2000
2,Antwerpen,Falafel Tof,Hoogstraat 32,"[Eethuis, Snack, 100% vegetarisch, Approved by...",2000


In [7]:
tags = pd.read_json('EVA_tags.json')

In [8]:
tags.head()

,tag
0,Eethuis
1,EVA voordeel
2,100% vegetarisch
3,Snack
4,Cateraar


In [9]:
all_data['full_address'] = all_data['street'] + ' ' + all_data['zipcode'] + ' ' + all_data['city']

In [10]:
all_data['latlon'] = all_data['full_address'].apply(lambda l: latlon_from_address(l))

In [11]:
all_data['found']=all_data['latlon'].apply(lambda p: p is None)
all_data.rename(columns={'found':'missing'}, inplace=True)

In [30]:
all_data.to_csv('EVA_restoswithlocations.csv', index=False)